# Forward differentiation

See

- [Simple forward-mode AD in Julia using Dual Numbers and Operator Overloading](https://www.youtube.com/watch?v=DfKKTE9XU0Q&list=PLISXH-iEM4JkHNH8Ji0ELDYcak7FD5gqa)

- [Simple (scalar) forward-mode Automatic Differentiation with DualNumbers and operator overloading](https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/adjoints_sensitivities_automatic_differentiation/scalar_forward_ad_with_dual_numbers.jl)



In [1]:
struct DualNumber{T}
    real::T;
    dual::T;
end

In [2]:
DualNumber(x) = DualNumber(x, zero(x))

DualNumber

In [3]:
f(x) = x * x
f_prime(x) = 2.0 * x
x_point = 3.0

3.0

In [4]:
f(x_point), f_prime(x_point)

(9.0, 6.0)

In [5]:
function pushforward(f, primal::Real, tangent::Real)
    input = DualNumber(primal, tangent)
    output = f(input)
    primal_out = output.real
    tangent_out = output.dual
    return primal_out, tangent_out
end

pushforward (generic function with 1 method)

In [6]:
function derivative(f, x::Real)
    v = one(x)
    _, df_dx = pushforward(f, x, v)
    return df_dx
end

derivative (generic function with 1 method)

In [7]:
try
    derivative(f, x_point)
catch e
    e
end

MethodError(*, (DualNumber{Float64}(3.0, 1.0), DualNumber{Float64}(3.0, 1.0)), 0x00000000000082cc)

$(f\,g)' = f\, g' + f'\,g$

In [8]:
Base.:*(x::DualNumber, y::DualNumber) =
    DualNumber(x.real * y.real, x.real * y.dual + x.dual * y.real)
Base.:*(x, y::DualNumber) =
    DualNumber(x) * y

In [9]:
pushforward(f, x_point, 10.0)

(9.0, 60.0)

In [10]:
derivative(f, x_point)

6.0

In [11]:
g(x) = 3.0 * x * x + sin(x)
g_prime(x) = 6.0 * x + cos(x)

g(x_point), g_prime(x_point)

(27.14112000805987, 17.010007503399553)

In [12]:
try
    derivative(g, x_point)
catch e
    e
end

MethodError(sin, (DualNumber{Float64}(3.0, 1.0),), 0x00000000000082d0)

- $(\sin x)' = \cos x$
- $y = f(g(x)) ⇒ y' = f'(g(x))\,g'(x) $

In [13]:
Base.sin(z::DualNumber) = DualNumber(sin(z.real), cos(z.real) * z.dual)

In [14]:
try
    derivative(g, x_point)
catch e
    e
end

MethodError(+, (DualNumber{Float64}(27.0, 18.0), DualNumber{Float64}(0.1411200080598672, -0.9899924966004454)), 0x00000000000082d1)

In [15]:
Base.:+(x::DualNumber, y::DualNumber) =
    DualNumber(x.real + y.real, x.dual + y.dual)
Base.:+(x, y::DualNumber) =
    DualNumber(x) * y


In [16]:
derivative(g, x_point)

17.010007503399553

See

- [Automatic Differentiation in 10 minutes with Julia](https://www.youtube.com/watch?v=vAp6nUMrKYg)

## Babylonian sqrt

Repeat $t ← (t + x/t)/2$ until $t$ converges to $\sqrt x$.

In [17]:
function babylonian(x; N=10)
    t = (1.0 + x) / 2.0
    for i = 2:N
        t = (t + x / t) / 2.0
    end
    t
end

babylonian (generic function with 1 method)

In [18]:
sqrt(2.0), babylonian(2.0)

(1.4142135623730951, 1.414213562373095)

In [19]:
sqrt_prime(x) = 0.5 / sqrt(x)
sqrt_prime(2.0)

0.35355339059327373

In [20]:
try
    derivative(babylonian, 2.0)
catch e
    e
end

MethodError(/, (DualNumber{Float64}(2.0, 1.0), 2.0), 0x00000000000082d7)

$(f/g)' = (f'g - fg')/g^2$

In [21]:
Base.:/(x::DualNumber, y::DualNumber) =
    DualNumber(x.real / y.real, (x.dual * y.real - x.real * y.dual) / (y.real * y.real))
Base.:/(x::DualNumber, y) =
    x / DualNumber(y)

In [22]:
derivative(babylonian, 2.0), sqrt_prime(2.0)

(0.35355339059327373, 0.35355339059327373)